# Run Multi-Agent Network model

This notebook guides you through running the multi-agent network model.

    Note: If you haven't already completed the setup, please run one-time-setup.ipynb first. It configures the required vector databases.

This notebook will read from existing vector databases and then initialize and execute the multi-agent network. It does not modify or rebuild the databases.

___________________________________________

## 1. Set Up API Keys

As with the setup notebook, use the cell below to enter your OpenAI API key. This will be stored as an environment variable for the current session only.

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ''


## 2. Read RAG databases

This cell loads the Chroma vector databases containing academic papers and their corresponding summaries, which were generated during `one-time-setup.ipynb`.

The locations of the persistent databases are defined using the `PAPER_PERSIST_DIR` and `SUMMARY_PERSIST_DIR` variables, specified as relative paths.



In [ ]:
# === Load Prebuilt Vector Databases === #

from model_resources.functions.vector_db import read_text_database

# Define relative paths to persisted Chroma vector databases
PAPER_PERSIST_DIR = "./model_databases/paperstore"
SUMMARY_PERSIST_DIR = "./model_databases/summarystore"

# Initialize the paper and summary vector stores
paperdb = read_text_database(PAPER_PERSIST_DIR)
summarydb = read_text_database(SUMMARY_PERSIST_DIR)

# Convert vector stores into retrievers for use in the RAG pipeline
paper_retriever = paperdb.as_retriever()
summary_retriever = summarydb.as_retriever()



## 3. Set Up Retriever Tools

This section initializes the Paper and Summary Retriever Tools used by the multi-agent network.


These retrievers are wrapped as tools using LangChain's create_retriever_tool, and then passed to a ToolExecutor for use in the agent workflow.

In [ ]:
from langchain.tools.retriever import create_retriever_tool
from langgraph.prebuilt import ToolExecutor
import model_resources.functions.multi_agent_network as man

# === Create Retriever Tools === #

paper_tool = create_retriever_tool(
    paper_retriever,
    "paper_retriever",
    "This tool gives access to the academic literature on deep eutectic electrolytes for zinc batteries. "
    "It can be used to extract detailed performance information about specific electrolyte components."
)

summary_tool = create_retriever_tool(
    summary_retriever,
    "summary_retriever",
    "This tool gives access to summaries that provide an overview of the field of deep eutectic electrolytes for zinc batteries. "
    "It is useful for determining whether a specific electrolyte composition has been tested in the literature."
)

# === Register Tools with ToolExecutor === #

tools = [paper_tool, summary_tool]
tool_executor = ToolExecutor(tools)

# === Inject ToolExecutor into Multi-Agent Network Module === #

man.tool_executor = tool_executor

# Expose the tool_node from the module for use in the graph
tool_node = man.tool_node

## 4. Set Up Agent Roles

This section creates two agents with distinct roles and capabilities:

- Scientist Agent: Uses the `paper_retriever` tool to extract detailed technical and performance information from academic literature.

- Principal Investigator (PI) Agent: Uses the `summary_retriever` tool to assess the research landscape, helping determine whether a given electrolyte composition has been previously studied.

Each agent is initialized with a custom system prompt (loaded from a text file) and wrapped into a callable node for use in the LangGraph network.

In [ ]:
import functools
from langchain.chat_models import ChatOpenAI
from model_resources.functions.multi_agent_network import create_agent, agent_node

# === Load System Prompts from Files === #

def load_agent_message(path: str) -> str:
    """Utility function to read an agent's system prompt from a file."""
    with open(path, "r", encoding="utf-8") as f:
        return f.read()

SCIENTIST_MSG_PATH = "model_resources/agent_messages/scientist_message.txt"
PI_MSG_PATH = "model_resources/agent_messages/PI_message.txt"

scientist_message = load_agent_message(SCIENTIST_MSG_PATH)
pi_message = load_agent_message(PI_MSG_PATH)

# === Initialize Language Model === #

llm = ChatOpenAI(
    model="gpt-4-0125-preview",
    temperature=0.7,
)

# === Create Scientist Agent and Node === #

scientist_agent = create_agent(
    llm=llm,
    tools=[paper_tool],
    system_message=scientist_message,
)

scientist_node = functools.partial(
    agent_node,
    agent=scientist_agent,
    name="Scientist"
)

# === Create Principal Investigator Agent and Node === #

PI_agent = create_agent(
    llm=llm,
    tools=[summary_tool],
    system_message=pi_message,
)

PI_node = functools.partial(
    agent_node,
    agent=PI_agent,
    name="Principal Investigator"
)

## 5. Build the Multi-Agent Workflow Graph

This step defines the multi-agent workflow using LangGraph.
Agents take turns invoking tools or passing control based on the message content.

- The Principal Investigator agent starts the interaction.

- Agents alternate based on the router's decision:

    - `continue`: passes control to the other agent

    - `call_tool`: invokes a tool

    - `end`: terminates the graph if a final answer is detected

The `tool_node` returns control to the agent who invoked the tool, determined by the sender field in the state.

In [ ]:
from langgraph.graph import END, StateGraph
from model_resources.functions.multi_agent_network import AgentState, tool_node, router

# === Initialize Workflow Graph === #

workflow = StateGraph(AgentState)

# === Add Agent and Tool Nodes === #

workflow.add_node("Scientist", scientist_node)
workflow.add_node("Principal Investigator", PI_node)
workflow.add_node("call_tool", tool_node)

# === Define Agent Transition Logic === #

workflow.add_conditional_edges(
    "Scientist",
    router,
    {
        "continue": "Principal Investigator",
        "call_tool": "call_tool",
        "end": END,
    },
)

workflow.add_conditional_edges(
    "Principal Investigator",
    router,
    {
        "continue": "Scientist",
        "call_tool": "call_tool",
        "end": END,
    },
)

# === Define Tool Return Routing === #

workflow.add_conditional_edges(
    "call_tool",
    # Route back to the original sender after the tool finishes
    lambda x: x["sender"],
    {
        "Scientist": "Scientist",
        "Principal Investigator": "Principal Investigator",
    },
)

# === Set Entry Point and Compile Graph === #

workflow.set_entry_point("Principal Investigator")
graph = workflow.compile()


## 6. Run the Multi-Agent Graph

This step streams the conversation through the compiled multi-agent graph using the initial user prompt.
Each agent takes turns reasoning, invoking tools, or concluding with a `FINAL ANSWER`.

- The input prompt is loaded from a file (`model_resources/prompts/initial_prompt.txt`)

- Output is streamed step-by-step to the console and saved to a timestamped .txt file in the results/ directory.

- The graph is limited to a maximum of 50 steps for safety and performance.

- You can run this notebook as many times as you'd like to generate new electrolyte compositions and exploration paths. 

In [ ]:
from datetime import datetime
from langchain_core.messages import HumanMessage
import json


# Load the initial prompt from file
PROMPT_PATH = "model_resources/prompts/initial_prompt.txt"
with open(PROMPT_PATH, "r", encoding="utf-8") as f:
    initial_prompt = f.read()

# Automatically generate a file name based on the current date and time
file_name = datetime.now().strftime("results/results_%Y%m%d_%H%M%S.txt")

with open(file_name, 'w', encoding="utf-8") as file:
    for s in graph.stream(
        {
            "messages": [
                HumanMessage(content=initial_prompt)
            ],
        },
        {"recursion_limit": 50},  # Maximum number of steps to take in the graph
    ):
        print(s)
        print("----")

        file.write(str(s) + '\n')
        file.write("----\n")
